# Eric González Caballero - MSc Big Data Analytics Thesis
## Forecasting the System Imbalance in the Spanish Electricity Market


### Notebook 04b - LGBM Model

This notebook aims to create a LGBM model to predict the system imbalances.

#### Library Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import lightgbm as lgb

#### Constants definition

In [2]:
input_path = "../data/curated/data_silver.csv"

#### Input

In [3]:
# Read data and name index column
df_full = pd.read_csv(input_path, index_col=0, parse_dates=True)

# Filter Data by date
df_full = df_full.loc['2019-01-01':]

# Shift column 'Signo del desvío' to first position
first_column = df_full.pop('Signo del desvío')
df_full.insert(0, 'Signo del desvío', first_column)

#Change data types to lower precission for faster training
new_types = {np.dtype(np.int64): np.int32, 
             np.dtype(np.float64): np.float32}

df_full = df_full.astype(df_full.dtypes.map(new_types).to_dict())

# Show dataframe
df_full

,Signo del desvío,Previsión diaria D+1 demanda,Previsión diaria D+1 fotovoltaica,Previsión diaria D+1 eólica,Precio mercado SPOT Diario,Signo_lag_24h,Signo_lag_48h,Holiday,Year,Month,Day,Weekday,Hour
DateIndex,,,,,,,,,,,,,
2019-01-01 00:00:00,-1.0,23753.000000,0.000000,3214.000000,66.879997,NaN,NaN,1,2019,1,1,1,0
2019-01-01 01:00:00,-1.0,23018.000000,0.000000,3222.000000,66.879997,NaN,NaN,1,2019,1,1,1,1
2019-01-01 02:00:00,-1.0,21808.000000,0.000000,3081.000000,66.000000,NaN,NaN,1,2019,1,1,1,2
2019-01-01 03:00:00,-1.0,20635.000000,0.000000,3069.000000,63.639999,NaN,NaN,1,2019,1,1,1,3
2019-01-01 04:00:00,-1.0,19824.000000,0.000000,2973.000000,58.849998,NaN,NaN,1,2019,1,1,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-10 19:00:00,NaN,31688.300781,4284.399902,4608.000000,153.149994,-1.0,1.0,0,2022,8,10,2,19
2022-08-10 20:00:00,NaN,31126.500000,1513.599976,4780.500000,166.500000,NaN,1.0,0,2022,8,10,2,20
2022-08-10 21:00:00,NaN,31363.500000,99.599998,5048.299805,171.679993,NaN,1.0,0,2022,8,10,2,21


In [4]:
# Derive dummies
n_in = 24*5
n_out = 36

df_full = df_full.drop('Signo_lag_24h', 1).drop('Signo_lag_48h', 1)

cols_original = df_full.columns[1:]
cols = [col + '(t)' for col in cols_original]

df_full.columns = ['Signo del desvío'] + list(cols)

for t in range(1, n_in + 1):
    for col in cols_original:
        df_full[col + f'(t-{t})'] = df_full[col + '(t)'].shift(t)
        
col = 'Signo del desvío'

for t in range(1, n_in + 1):
    df_full[col + f'(t-{t})'] = df_full[col].shift(t)

# If forecasts from future hours want to be added, similar accuracy results    
# for i in range(1,37):
#     df_full['Demanda in ' + str(i) + ' hours'] = df_full['Previsión diaria D+1 demanda(t)'].shift(-i)
#     df_full['Fotovoltaica in ' + str(i) + ' hours'] = df_full['Previsión diaria D+1 fotovoltaica(t)'].shift(-i)
#     df_full['Eólica in ' + str(i) + ' hours'] = df_full['Previsión diaria D+1 eólica(t)'].shift(-i)
#     df_full['Precio in ' + str(i) + ' hours'] = df_full['Precio mercado SPOT Diario(t)'].shift(-i)
#     df_full['Holiday in ' + str(i) + ' hours'] = df_full['Holiday(t)'].shift(-i)
#     df_full['Year in ' + str(i) + ' hours'] = df_full['Year(t)'].shift(-i)
#     df_full['Month in ' + str(i) + ' hours'] = df_full['Month(t)'].shift(-i)
#     df_full['Day in ' + str(i) + ' hours'] = df_full['Day(t)'].shift(-i)
#     df_full['Weekday in ' + str(i) + ' hours'] = df_full['Weekday(t)'].shift(-i)
#     df_full['Hour in ' + str(i) + ' hours'] = df_full['Hour(t)'].shift(-i)    
    
for t in range(1, n_out + 1):
    df_full[col + f'(t+{t})'] = df_full[col].shift(-t)       
    
df_full = df_full.drop('Signo del desvío', 1)

In [5]:
df_full

,Previsión diaria D+1 demanda(t),Previsión diaria D+1 fotovoltaica(t),Previsión diaria D+1 eólica(t),Precio mercado SPOT Diario(t),Holiday(t),Year(t),Month(t),Day(t),Weekday(t),Hour(t),...,Signo del desvío(t+27),Signo del desvío(t+28),Signo del desvío(t+29),Signo del desvío(t+30),Signo del desvío(t+31),Signo del desvío(t+32),Signo del desvío(t+33),Signo del desvío(t+34),Signo del desvío(t+35),Signo del desvío(t+36)
DateIndex,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00,23753.000000,0.000000,3214.000000,66.879997,1,2019,1,1,1,0,...,-1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0
2019-01-01 01:00:00,23018.000000,0.000000,3222.000000,66.879997,1,2019,1,1,1,1,...,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,1.0
2019-01-01 02:00:00,21808.000000,0.000000,3081.000000,66.000000,1,2019,1,1,1,2,...,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,1.0,-1.0
2019-01-01 03:00:00,20635.000000,0.000000,3069.000000,63.639999,1,2019,1,1,1,3,...,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0
2019-01-01 04:00:00,19824.000000,0.000000,2973.000000,58.849998,1,2019,1,1,1,4,...,1.0,1.0,1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-10 19:00:00,31688.300781,4284.399902,4608.000000,153.149994,0,2022,8,10,2,19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-08-10 20:00:00,31126.500000,1513.599976,4780.500000,166.500000,0,2022,8,10,2,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-08-10 21:00:00,31363.500000,99.599998,5048.299805,171.679993,0,2022,8,10,2,21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Check column datatypes
df_full.dtypes

Previsión diaria D+1 demanda(t)         float32
Previsión diaria D+1 fotovoltaica(t)    float32
Previsión diaria D+1 eólica(t)          float32
Precio mercado SPOT Diario(t)           float32
Holiday(t)                                int32
                                         ...   
Signo del desvío(t+32)                  float32
Signo del desvío(t+33)                  float32
Signo del desvío(t+34)                  float32
Signo del desvío(t+35)                  float32
Signo del desvío(t+36)                  float32
Length: 1366, dtype: object

In [7]:
# Split data into: 
# df -> Full train/val/test data (excluding first NaNs due to column shift)

df = df_full[72:-36] # Before last 36 hours  // 72 instead of 48h due to 48+24 lag

In [8]:
df

,Previsión diaria D+1 demanda(t),Previsión diaria D+1 fotovoltaica(t),Previsión diaria D+1 eólica(t),Precio mercado SPOT Diario(t),Holiday(t),Year(t),Month(t),Day(t),Weekday(t),Hour(t),...,Signo del desvío(t+27),Signo del desvío(t+28),Signo del desvío(t+29),Signo del desvío(t+30),Signo del desvío(t+31),Signo del desvío(t+32),Signo del desvío(t+33),Signo del desvío(t+34),Signo del desvío(t+35),Signo del desvío(t+36)
DateIndex,,,,,,,,,,,,,,,,,,,,,
2019-01-04 00:00:00,27702.000000,0.000000,4175.000000,67.199997,0,2019,1,4,4,0,...,-1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0
2019-01-04 01:00:00,25236.000000,0.000000,4089.000000,63.750000,0,2019,1,4,4,1,...,-1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0
2019-01-04 02:00:00,23829.000000,0.000000,3991.000000,58.889999,0,2019,1,4,4,2,...,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,-1.0
2019-01-04 03:00:00,23185.000000,0.000000,3897.000000,54.939999,0,2019,1,4,4,3,...,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,-1.0,-1.0
2019-01-04 04:00:00,23013.000000,0.000000,3785.000000,54.849998,0,2019,1,4,4,4,...,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-09 07:00:00,26012.800781,281.500000,5303.299805,152.740005,0,2022,8,9,1,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-08-09 08:00:00,27288.500000,2516.300049,4604.500000,152.850006,0,2022,8,9,1,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-08-09 09:00:00,29001.000000,6150.399902,3798.500000,145.490005,0,2022,8,9,1,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Split Train/Validation/Test data

In [9]:
# Train: 70% 
# Validation: 0 %; No validation because no hyperparrameter tuning implemented
# Test: 30%

perc_train = 0.70 
perc_val = 0
perc_test = 0.30

n = len(df)

train_df = df[0:int(n*perc_train)]
val_df = df[int(n*perc_train):int(n*(perc_train+perc_val))]
test_df = df[int(n*(perc_train+perc_val)):]

In [10]:
#See shapes
print(df.shape)
print(train_df.shape)
print(val_df.shape)
print(test_df.shape)

(31523, 1366)
(22066, 1366)
(0, 1366)
(9457, 1366)


## LightGBM model

In [11]:
train_df

,Previsión diaria D+1 demanda(t),Previsión diaria D+1 fotovoltaica(t),Previsión diaria D+1 eólica(t),Precio mercado SPOT Diario(t),Holiday(t),Year(t),Month(t),Day(t),Weekday(t),Hour(t),...,Signo del desvío(t+27),Signo del desvío(t+28),Signo del desvío(t+29),Signo del desvío(t+30),Signo del desvío(t+31),Signo del desvío(t+32),Signo del desvío(t+33),Signo del desvío(t+34),Signo del desvío(t+35),Signo del desvío(t+36)
DateIndex,,,,,,,,,,,,,,,,,,,,,
2019-01-04 00:00:00,27702.0,0.000000,4175.0,67.199997,0,2019,1,4,4,0,...,-1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0
2019-01-04 01:00:00,25236.0,0.000000,4089.0,63.750000,0,2019,1,4,4,1,...,-1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0
2019-01-04 02:00:00,23829.0,0.000000,3991.0,58.889999,0,2019,1,4,4,2,...,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,-1.0
2019-01-04 03:00:00,23185.0,0.000000,3897.0,54.939999,0,2019,1,4,4,3,...,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,-1.0,-1.0
2019-01-04 04:00:00,23013.0,0.000000,3785.0,54.849998,0,2019,1,4,4,4,...,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-11 06:00:00,21702.0,44.599998,3249.0,98.559998,0,2021,7,11,6,6,...,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,1.0,1.0
2021-07-11 07:00:00,21467.0,459.299988,3279.0,98.000000,0,2021,7,11,6,7,...,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,1.0,1.0,1.0
2021-07-11 08:00:00,22376.0,2187.699951,3052.0,93.500000,0,2021,7,11,6,8,...,1.0,1.0,1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0


In [12]:
x_train = train_df[train_df.columns[:-36]][n_in:]
y_train = train_df[train_df.columns[-36:]][n_in:]

x_test = test_df[test_df.columns[:-36]][:-36]
y_test = test_df[test_df.columns[-36:]][:-36]

In [13]:
# By default LGBM only outputs one binary values, we need the series of 36 values, so MultiOutputClassifier is used
# https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html
# https://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html#sklearn.multioutput.MultiOutputClassifier

from sklearn.multioutput import MultiOutputClassifier

model = MultiOutputClassifier(lgb.LGBMClassifier(random_state=42))

model.fit(x_train, y_train)

MultiOutputClassifier(estimator=LGBMClassifier(random_state=42))

In [14]:
y_pred = model.predict(x_test)

In [15]:
y_pred.shape

(9421, 36)

In [16]:
from sklearn.metrics import accuracy_score

acc_list = []

for i in range(0, y_pred.shape[0]):
    acc = accuracy_score(y_test.values[i], y_pred[i])
    acc_list.append(acc)

In [17]:
# Average accuracy
np.mean(acc_list)

0.8284742124568045

In [18]:
# 4 Highest features by importance

feat_impts = [] 
for clf in model.estimators_:
    feat_impts.append(clf.feature_importances_)

show_feats = 5
args = np.argpartition(np.mean(feat_impts, axis=0), -show_feats)[-show_feats:]

In [19]:
for i in range(0,show_feats):
    print(x_test.columns[args[i]])

Previsión diaria D+1 demanda(t-3)
Previsión diaria D+1 demanda(t-2)
Previsión diaria D+1 demanda(t-1)
Hour(t)
Previsión diaria D+1 demanda(t)


In [20]:
# Scoring future forecast
x_forec = test_df[test_df.columns[0:-36]][-1:]
y_forec = model.predict(x_forec)

In [21]:
x_forec

,Previsión diaria D+1 demanda(t),Previsión diaria D+1 fotovoltaica(t),Previsión diaria D+1 eólica(t),Precio mercado SPOT Diario(t),Holiday(t),Year(t),Month(t),Day(t),Weekday(t),Hour(t),...,Signo del desvío(t-111),Signo del desvío(t-112),Signo del desvío(t-113),Signo del desvío(t-114),Signo del desvío(t-115),Signo del desvío(t-116),Signo del desvío(t-117),Signo del desvío(t-118),Signo del desvío(t-119),Signo del desvío(t-120)
DateIndex,,,,,,,,,,,,,,,,,,,,,
2022-08-09 11:00:00,31323.0,9565.200195,3271.800049,130.0,0,2022,8,9,1,11,...,1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [22]:
# Final predictions
y_forec[-1]

array([ 1.,  1., -1., -1., -1.,  1., -1., -1.,  1.,  1., -1., -1., -1.,
       -1., -1., -1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1., -1.,  1.,  1.,  1., -1., -1.], dtype=float32)